# NB GLM test

In [1]:
import multiprocessing
import warnings
from math import floor
from pathlib import Path
from typing import List
from typing import Literal
from typing import Optional
from typing import Tuple
from typing import Union
from typing import cast

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.linalg import solve  # type: ignore
from scipy.optimize import minimize  # type: ignore
from scipy.special import gammaln  # type: ignore
from scipy.special import polygamma  # type: ignore
from scipy.stats import norm  # type: ignore
from sklearn.linear_model import LinearRegression  # type: ignore

In [3]:
#generate count data for a single gene (vector), sample 
#counts = [511, 1783, 241, 1129, 1302, 2204, 3888, 5035, 236, 468, 1424, 482, 842, 1145, 1261, 1661, 2712, 1707, 1125, 3832]
counts = [499, 564, 88, 1687, 2179, 768, 725, 911, 520, 1259, 150, 858, 1946, 38, 4813, 1808, 2308, 118, 1318, 380]
counts = np.array(counts)
counts 

array([ 499,  564,   88, 1687, 2179,  768,  725,  911,  520, 1259,  150,
        858, 1946,   38, 4813, 1808, 2308,  118, 1318,  380])

In [4]:
# simulate CN-induced differential gene expression
#cnv = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0.5, 1, 1, 0.5, 1, 0.5, 1, 0.5, 1, 0.5]
cnv = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 5, 5, 3, 5, 5, 5, 5]
cnv = np.array(cnv)
cnv = cnv/2
counts = counts * cnv
counts

array([  499. ,   564. ,    88. ,  1687. ,  2179. ,   768. ,   725. ,
         911. ,   520. ,  1259. ,   300. ,  1287. ,  2919. ,    95. ,
       12032.5,  2712. ,  5770. ,   295. ,  3295. ,   950. ])

In [5]:
#Generate dispersion data
disp = 0.6949248
alpha = disp
alpha

0.6949248

In [6]:
#Generate real calculated sf data (edgeR)
size_factors = np.array([ 1.1367718, 1.0516453, 1.0494177, 1.1070994, 1.2151661, 1.2099502, 1.0194880, 1.0139071,
                         1.0614114, 0.9878240, 0.9059977, 0.8133267, 1.0104569, 0.9603320, 0.9471832, 0.9393154, 1.0117350,
                         0.9449861, 0.9741997, 0.7625332])
size_factors

array([1.1367718, 1.0516453, 1.0494177, 1.1070994, 1.2151661, 1.2099502,
       1.019488 , 1.0139071, 1.0614114, 0.987824 , 0.9059977, 0.8133267,
       1.0104569, 0.960332 , 0.9471832, 0.9393154, 1.011735 , 0.9449861,
       0.9741997, 0.7625332])

In [7]:
X = {'condition': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
X = pd.DataFrame(X, index = ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8', 'sample9', 'sample10',
                            'sample11', 'sample12', 'sample13', 'sample14', 'sample15', 'sample16', 'sample17', 'sample18', 'sample19', 'sample20'])
X.insert(0, "intercept", 1)
X = np.array(X)
design_matrix = X

In [8]:
Q, R = np.linalg.qr(X)
y = np.log((counts / cnv) / size_factors + 0.1)
y

array([6.08464139, 6.28488481, 4.43029319, 7.32902926, 7.49179633,
       6.45336806, 6.56701172, 6.8008429 , 6.19443338, 7.15040223,
       5.10995762, 6.96132129, 7.56318058, 3.68058638, 8.5333583 ,
       7.56263246, 7.73251378, 4.8280702 , 7.21008359, 6.21148113])

In [11]:
beta_init = solve(R, Q.T @ y)
beta = beta_init


In [14]:
beta

array([6.47867033, 0.06064821])

In [17]:
min_mu = 0.5
mu = np.maximum(cnv * size_factors[:, None] * np.exp(design_matrix @ beta), min_mu)

array([[ 740.15736381,  740.15736381,  740.15736381,  740.15736381,
         740.15736381,  740.15736381,  740.15736381,  740.15736381,
         740.15736381,  740.15736381, 1572.87149599, 1179.65362199,
        1179.65362199, 1966.08936999, 1966.08936999, 1179.65362199,
        1966.08936999, 1966.08936999, 1966.08936999, 1966.08936999],
       [ 684.7311069 ,  684.7311069 ,  684.7311069 ,  684.7311069 ,
         684.7311069 ,  684.7311069 ,  684.7311069 ,  684.7311069 ,
         684.7311069 ,  684.7311069 , 1455.08792201, 1091.31594151,
        1091.31594151, 1818.85990252, 1818.85990252, 1091.31594151,
        1818.85990252, 1818.85990252, 1818.85990252, 1818.85990252],
       [ 683.28070627,  683.28070627,  683.28070627,  683.28070627,
         683.28070627,  683.28070627,  683.28070627,  683.28070627,
         683.28070627,  683.28070627, 1452.00574796, 1089.00431097,
        1089.00431097, 1815.00718495, 1815.00718495, 1089.00431097,
        1815.00718495, 1815.00718495, 1815.007

### CN normalized model

### Test GLM

In [9]:
def nb_nll(
    counts: np.ndarray, mu: np.ndarray, alpha: Union[float, np.ndarray]
) -> Union[float, np.ndarray]:
    n = len(counts)
    alpha_neg1 = 1 / alpha
    logbinom = gammaln(counts + alpha_neg1) - gammaln(counts + 1) - gammaln(alpha_neg1)
    if hasattr(alpha, "__len__") and len(alpha) > 1:
        return (
            alpha_neg1 * np.log(alpha)
            - logbinom
            + (counts + alpha_neg1) * np.log(mu + alpha_neg1)
            - (counts * np.log(mu))
        ).sum(0)
    else:
        return (
            n * alpha_neg1 * np.log(alpha)
            + (
                -logbinom
                + (counts + alpha_neg1) * np.log(alpha_neg1 + mu)
                - counts * np.log(mu)
            ).sum()
        )

In [10]:
def vec_nb_nll(counts: np.ndarray, mu: np.ndarray, alpha: np.ndarray) -> np.ndarray:
    n = len(counts)
    alpha_neg1 = 1 / alpha
    logbinom = (
        gammaln(counts[:, None] + alpha_neg1)
        - gammaln(counts + 1)[:, None]
        - gammaln(alpha_neg1)
    )

    if len(mu.shape) == 1:
        return n * alpha_neg1 * np.log(alpha) + (
            -logbinom
            + (counts[:, None] + alpha_neg1) * np.log(mu[:, None] + alpha_neg1)
            - (counts * np.log(mu))[:, None]
        ).sum(0)
    else:
        return n * alpha_neg1 * np.log(alpha) + (
            -logbinom
            + (counts[:, None] + alpha_neg1) * np.log(mu + alpha_neg1)
            - (counts[:, None] * np.log(mu))
        ).sum(0)

In [11]:
def grid_fit_beta(
    counts: np.ndarray,
    size_factors: np.ndarray,
    cnv: np.ndarray,
    design_matrix: np.ndarray,
    disp: float,
    min_mu: float = 0.5,
    grid_length: int = 60,
    min_beta: float = -30,
    max_beta: float = 30,
) -> np.ndarray:
    
    x_grid = np.linspace(min_beta, max_beta, grid_length)
    y_grid = np.linspace(min_beta, max_beta, grid_length)
    ll_grid = np.zeros((grid_length, grid_length))

    def loss(beta: np.ndarray) -> np.ndarray:
        # closure to minimize
        design_matrix_t = design_matrix.T
        offset = size_factors + cnv
        mu = np.maximum(np.log(offset[:, None]) + np.exp(design_matrix @ beta), min_mu)
        return vec_nb_nll(counts, mu, disp) + 0.5 * (1e-6 * beta**2).sum(1)

    for i, x in enumerate(x_grid):
        ll_grid[i, :] = loss(np.array([[x, y] for y in y_grid]))

    min_idxs = np.unravel_index(np.argmin(ll_grid, axis=None), ll_grid.shape)
    delta = x_grid[1] - x_grid[0]

    fine_x_grid = np.linspace(
        x_grid[min_idxs[0]] - delta, x_grid[min_idxs[0]] + delta, grid_length
    )

    fine_y_grid = np.linspace(
        y_grid[min_idxs[1]] - delta,
        y_grid[min_idxs[1]] + delta,
        grid_length,
    )

    for i, x in enumerate(fine_x_grid):
        ll_grid[i, :] = loss(np.array([[x, y] for y in fine_y_grid]))

    min_idxs = np.unravel_index(np.argmin(ll_grid, axis=None), ll_grid.shape)
    beta = np.array([fine_x_grid[min_idxs[0]], fine_y_grid[min_idxs[1]]])
    return beta

In [15]:
def irls_glm(
    counts: np.ndarray,
    size_factors: np.ndarray,
    design_matrix: np.ndarray,
    cnv: np.ndarray,
    disp: float,
    min_mu: float = 0.5,
    beta_tol: float = 1e-8,
    min_beta: float = -30,
    max_beta: float = 30,
    optimizer: Literal["BFGS", "L-BFGS-B"] = "L-BFGS-B",
    maxiter: int = 250,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, bool]:

    assert optimizer in ["BFGS", "L-BFGS-B"]
    
    X = design_matrix
    num_vars = design_matrix.shape[1]
    
    # if full rank, estimate initial betas for IRLS below
    if np.linalg.matrix_rank(X) == num_vars:
        Q, R = np.linalg.qr(X)
        y = np.log((counts/cnv)/size_factors + 0.1)
        #y = np.log(counts / size_factors + 0.1)
        beta_init = solve(R, Q.T @ y)
        beta = beta_init

    else:  # Initialise intercept with log base mean
        beta_init = np.zeros(num_vars)
        #beta_init[0] = np.log(counts / size_factors).mean()
        beta_init[0] = np.log((counts / cnv) / size_factors).mean()
        beta = beta_init
        
    dev = 1000.0
    dev_ratio = 1.0

    ridge_factor = np.diag(np.repeat(1e-6, num_vars))
    mu = np.maximum(np.log(cnv) + np.log(size_factors) + (np.exp(X @ beta)), min_mu)
    #mu = np.maximum(cnv * size_factors * np.exp(X @ beta), min_mu)
    #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
    
    converged = True
    i = 0
    while dev_ratio > beta_tol:
        W = mu / (1.0 + mu * disp)
        z = np.log((mu / cnv)/size_factors) + (counts - mu) / mu
        #z = np.log(mu / size_factors) + (counts - mu) / mu
        H = (X.T * W) @ X + ridge_factor
        beta_hat = solve(H, X.T @ (W * z), assume_a="pos")
        i += 1

        if sum(np.abs(beta_hat) > max_beta) > 0 or i >= maxiter:
            # If IRLS starts diverging, use L-BFGS-B
            def f(beta: np.ndarray) -> float:
                # closure to minimize
                mu_ = np.maximum(np.log(cnv) + np.log(size_factors) + np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(cnv * size_factors * np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X @ beta), min_mu)
                
                return nb_nll(counts, mu_, disp) + 0.5 * (ridge_factor @ beta**2).sum()

            def df(beta: np.ndarray) -> np.ndarray:
                mu_ = np.maximum(np.log(cnv) + np.log(size_factors) + np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(cnv * size_factors * np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X @ beta), min_mu)
                return (
                    -X.T @ counts
                    + ((1 / disp + counts) * mu_ / (1 / disp + mu_)) @ X
                    + ridge_factor @ beta
                )

            res = minimize(
                f,
                beta_init,
                jac=df,
                method=optimizer,
                bounds=(
                    [(min_beta, max_beta)] * num_vars
                    if optimizer == "L-BFGS-B"
                    else None
                ),
            )

            beta = res.x
            mu = np.maximum(np.log(cnv) + np.log(size_factors) + np.exp(X @ beta), min_mu)
            #mu = np.maximum(cnv * size_factors * np.exp(X @ beta), min_mu)
            #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
            converged = res.success

            if not res.success and num_vars <= 2:
                beta = grid_fit_beta(
                    counts,
                    size_factors,
                    cnv,
                    X,
                    disp,
                )
                mu = np.maximum(np.log(cnv) + np.log(size_factors) + np.exp(X @ beta), min_mu)
                #mu = np.maximum(cnv * size_factors * np.exp(X @ beta), min_mu)
                #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu) 
            break

        beta = beta_hat
        mu = np.maximum(np.log(cnv) + np.log(size_factors) + np.exp(X @ beta), min_mu)
        #mu = np.maximum(cnv * size_factors * np.exp(X @ beta), min_mu)
        #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
        # Compute deviation
        old_dev = dev
        # Replaced deviation with -2 * nll, as in the R code
        dev = -2 * nb_nll(counts, mu, disp)
        dev_ratio = np.abs(dev - old_dev) / (np.abs(dev) + 0.1)

    # Compute H diagonal (useful for Cook distance outlier filtering)
    W = mu / (1.0 + mu * disp)
    W_sq = np.sqrt(W)
    XtWX = (X.T * W) @ X + ridge_factor
    H = W_sq * np.diag(X @ np.linalg.inv(XtWX) @ X.T) * W_sq
    # Return an UNthresholded mu (as in the R code)
    # Previous quantities are estimated with a threshold though
    mu = np.log(cnv) + np.log(size_factors) + np.exp(X @ beta)
    #mu = cnv * size_factors * np.exp(X @ beta)
    #mu = size_factors * np.exp(X @ beta)

    #print("Beta parameters:", beta), 
    #print("Estimated mean:", np.array(mu)), 
    #print("H:", np.array(H)),
    #print("Convergence:", converged)
    
    return beta, mu, H, converged

In [64]:
# Classical GLM (sf) CN induced DGE (CN amplifications)
irls_glm(counts, size_factors, design_matrix, disp)

(array([6.73161842, 1.3119075 ]),
 array([ 953.18681752,  881.80797295,  879.94012317,  928.3064145 ,
        1018.92069069, 1014.54714173,  854.84397327,  850.16437064,
         889.99687927,  828.29360724, 2820.88913364, 2532.3512964 ,
        3146.13038115, 2990.06289253, 2949.1231561 , 2924.62619378,
        3150.10983761, 2942.28232691, 3033.24097591, 2374.20207349]),
 array([0.10000788, 0.09999568, 0.09999534, 0.10000384, 0.10001761,
        0.100017  , 0.09999054, 0.09998962, 0.09999718, 0.09998516,
        0.09999921, 0.09999341, 0.10000449, 0.1000021 , 0.10000143,
        0.10000102, 0.10000454, 0.10000132, 0.10000278, 0.09998962]),
 True)

In [81]:
# CN normalized GLM (CN amplifications)
irls_glm(counts, size_factors, design_matrix, cnv, disp)

(array([6.73161837, 0.54422075]),
 array([ 953.18676659,  881.80792583,  879.94007615,  928.3063649 ,
        1018.92063625, 1014.54708752,  854.8439276 ,  850.16432521,
         889.99683172,  828.29356298, 2618.26664635, 1762.84070964,
        2190.10953243, 3469.11096604, 3421.61213619, 2035.91425968,
        3654.79957268, 3413.67531465, 3519.20675598, 2754.58100541]),
 array([0.10000788, 0.09999568, 0.09999534, 0.10000384, 0.10001761,
        0.100017  , 0.09999054, 0.09998962, 0.09999718, 0.09998516,
        0.09999815, 0.0999715 , 0.09998741, 0.10001162, 0.10001105,
        0.09998244, 0.10001373, 0.10001095, 0.10001221, 0.10000087]),
 True)

In [100]:
# Classical GLM (sf) CN induced DGE (CN deletions)
irls_glm(counts, size_factors, design_matrix, disp)

(array([ 6.73161832, -0.24897681]),
 array([ 953.18671396,  881.80787715,  879.94002757,  928.30631365,
        1018.92058   , 1014.54703151,  854.8438804 ,  850.16427828,
         889.99678258,  828.29351725,  592.24653637,  531.66792919,
         660.53103575,  627.76461879,  619.1693086 ,  614.02616387,
         661.36652386,  617.73307442,  636.8298706 ,  498.46445147]),
 array([0.10000788, 0.09999568, 0.09999534, 0.10000384, 0.10001761,
        0.100017  , 0.09999054, 0.09998962, 0.09999718, 0.09998516,
        0.09999629, 0.09996868, 0.10002135, 0.10001   , 0.10000683,
        0.10000489, 0.10002163, 0.10000629, 0.10001326, 0.09995071]),
 True)

In [102]:
# CN normalized GLM (CN deletions)
irls_glm(counts, size_factors, design_matrix, cnv, disp)

(array([6.73161837, 0.5449547 ]),
 array([ 953.18676664,  881.80792588,  879.9400762 ,  928.30636494,
        1018.9206363 , 1014.54708758,  854.84392764,  850.16432526,
         889.99683176,  828.29356302,  327.52362671,  588.04500383,
         730.57251364,  347.16580349,  684.82486616,  339.56817598,
         731.49659534,  341.61816819,  704.35812118,  275.66034566]),
 array([0.10000788, 0.09999568, 0.09999534, 0.10000384, 0.10001761,
        0.100017  , 0.09999054, 0.09998962, 0.09999718, 0.09998516,
        0.09988835, 0.10008231, 0.10012999, 0.09991308, 0.10011685,
        0.09990385, 0.10013024, 0.09990638, 0.10012267, 0.09980621]),
 True)

In [16]:
irls_glm(counts, size_factors, design_matrix, cnv, disp)

(array([6.74790044, 0.73913047]),
 array([ 852.39568755,  852.31785095,  852.3157305 ,  852.3692385 ,
         852.46237584,  852.45807426,  852.2867956 ,  852.28130634,
         852.32709459,  852.25524433, 1785.33956641, 1784.94398044,
        1785.16100545, 1785.62095225, 1785.60716572, 1785.08799888,
        1785.67309515, 1785.60484341, 1785.63528951, 1785.39031924]),
 array([0.1       , 0.09999998, 0.09999998, 0.09999999, 0.10000001,
        0.10000001, 0.09999997, 0.09999997, 0.09999998, 0.09999997,
        0.09999999, 0.09999997, 0.09999998, 0.1       , 0.1       ,
        0.09999998, 0.10000001, 0.1       , 0.1       , 0.09999999]),
 True)

Using library size variable

In [139]:
def irls(
    counts: np.ndarray,
    size_factors: np.ndarray,
    design_matrix: np.ndarray,
    cnv: np.ndarray,
    lib_size: np.ndarray,
    disp: float,
    min_mu: float = 0.5,
    beta_tol: float = 1e-8,
    min_beta: float = -30,
    max_beta: float = 30,
    optimizer: Literal["BFGS", "L-BFGS-B"] = "L-BFGS-B",
    maxiter: int = 250,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, bool]:

    assert optimizer in ["BFGS", "L-BFGS-B"]
    
    X = design_matrix
    num_vars = design_matrix.shape[1]
    
    # if full rank, estimate initial betas for IRLS below
    if np.linalg.matrix_rank(X) == num_vars:
        Q, R = np.linalg.qr(X)
        lib_size = lib_size * size_factors
        lib_size = np.log(lib_size) 
        y = np.log(counts - lib_size - cnv + 0.1)
        #y = np.log(counts / size_factors + 0.1)
        beta_init = solve(R, Q.T @ y)
        beta = beta_init

    else:  # Initialise intercept with log base mean
        beta_init = np.zeros(num_vars)
        #beta_init[0] = np.log(counts / size_factors).mean()
        beta_init[0] = np.log(counts - lib_size - cnv).mean()
        beta = beta_init
        
    dev = 1000.0
    dev_ratio = 1.0

    ridge_factor = np.diag(np.repeat(1e-6, num_vars))
    mu = np.maximum(lib_size + cnv + np.exp(X @ beta), min_mu)
    #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
    
    converged = True
    i = 0
    while dev_ratio > beta_tol:
        W = mu / (1.0 + mu * disp)
        z = np.log(mu - lib_size - cnv) + (counts - mu) / mu
        #z = np.log(mu / size_factors) + (counts - mu) / mu
        H = (X.T * W) @ X + ridge_factor
        beta_hat = solve(H, X.T @ (W * z), assume_a="pos")
        i += 1

        if sum(np.abs(beta_hat) > max_beta) > 0 or i >= maxiter:
            # If IRLS starts diverging, use L-BFGS-B
            def f(beta: np.ndarray) -> float:
                # closure to minimize
                mu_ = np.maximum(lib_size + cnv + np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X @ beta), min_mu)
                
                return nb_nll(counts, mu_, disp) + 0.5 * (ridge_factor @ beta**2).sum()

            def df(beta: np.ndarray) -> np.ndarray:
                mu_ = np.maximum(lib_size + cnv + np.exp(X @ beta), min_mu)
                #mu_ = np.maximum(size_factors * np.exp(X @ beta), min_mu)
                return (
                    -X.T @ counts
                    + ((1 / disp + counts) * mu_ / (1 / disp + mu_)) @ X
                    + ridge_factor @ beta
                )

            res = minimize(
                f,
                beta_init,
                jac=df,
                method=optimizer,
                bounds=(
                    [(min_beta, max_beta)] * num_vars
                    if optimizer == "L-BFGS-B"
                    else None
                ),
            )

            beta = res.x
            mu = np.maximum(lib_size + cnv + np.exp(X @ beta), min_mu)
            #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
            converged = res.success

            if not res.success and num_vars <= 2:
                beta = grid_fit_beta(
                    counts,
                    size_factors,
                    cnv,
                    X,
                    disp,
                )
                mu = np.maximum(lib_size + cnv + np.exp(X @ beta), min_mu)
                #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu) 
            break

        beta = beta_hat
        mu = np.maximum(lib_size + cnv + np.exp(X @ beta), min_mu)
        #mu = np.maximum(size_factors * np.exp(X @ beta), min_mu)
        # Compute deviation
        old_dev = dev
        # Replaced deviation with -2 * nll, as in the R code
        dev = -2 * nb_nll(counts, mu, disp)
        dev_ratio = np.abs(dev - old_dev) / (np.abs(dev) + 0.1)

    # Compute H diagonal (useful for Cook distance outlier filtering)
    W = mu / (1.0 + mu * disp)
    W_sq = np.sqrt(W)
    XtWX = (X.T * W) @ X + ridge_factor
    H = W_sq * np.diag(X @ np.linalg.inv(XtWX) @ X.T) * W_sq
    # Return an UNthresholded mu (as in the R code)
    # Previous quantities are estimated with a threshold though
    mu = lib_size + cnv + np.exp(X @ beta)
    #mu = size_factors * np.exp(X @ beta)

    #print("Beta parameters:", beta), 
    #print("Estimated mean:", np.array(mu)), 
    #print("H:", np.array(H)),
    #print("Convergence:", converged)
    
    return beta, mu, H, converged

In [116]:
irls(counts, size_factors, design_matrix, lib_size, disp)

(array([6.81123249, 1.17953719]),
 array([ 920.06905607,  919.99170944,  920.10577947,  919.98583924,
         920.04209008,  920.011077  ,  919.96273378,  919.98565247,
         919.94105733,  919.96183234, 2965.50461773, 2965.52506473,
        2965.55517745, 2965.56268777, 2965.49755917, 2965.66277913,
        2965.61372518, 2965.65639418, 2965.56050622, 2965.46684246]),
 array([0.1       , 0.09999998, 0.1       , 0.09999998, 0.09999999,
        0.09999999, 0.09999998, 0.09999998, 0.09999998, 0.09999998,
        0.09999999, 0.09999999, 0.09999999, 0.09999999, 0.09999999,
        0.09999999, 0.09999999, 0.09999999, 0.09999999, 0.09999999]),
 True)

In [1]:
#irls(counts, size_factors, design_matrix, lib_size, cnv, disp)